In [1]:
setwd('/fs/cbcb-lab/rob/students/noor/Uncertainity/treeTerminusScripts')
load("environment/brain_sim_nodtu/mode=gc_bias/mapDf.RData")
# load("environment/brain_sim_nodtu/mode=gc_bias/post_type=gibbs_nrep=100_tf=100/mean_cons/yAggCons.RData")
# load("environment/brain_sim_nodtu/mode=gc_bias/post_type=gibbs_nrep=100_tf=100/mean_cons/yAggCor.RData")
load("environment/brain_sim_nodtu/mode=gc_bias/post_type=gibbs_nrep=100_tf=100/mean_cons/yAggMeanInf.RData")
load("environment/brain_sim_nodtu/mode=gc_bias/post_type=gibbs_nrep=100_tf=100/mean_cons/yAggMeanInf0.RData")
load("environment/brain_sim_nodtu/mode=gc_bias/post_type=gibbs_nrep=100_tf=100/mean_cons/yAggThrNS.RData")
load("environment/brain_sim_nodtu/mode=gc_bias/post_type=gibbs_nrep=100_tf=100/mean_cons/treeCons.RData")
load("environment/brain_sim_nodtu/mode=gc_bias/post_type=gibbs_nrep=100_tf=100/mean_cons/treeCor.RData")
load("environment/brain_sim_nodtu/mode=gc_bias/post_type=gibbs_nrep=100_tf=100/mean_cons/treeMC.RData")
load("environment/brain_sim_nodtu/mode=gc_bias/post_type=gibbs_nrep=100_tf=100/mean_cons/treeMC0.RData")
load("environment/brain_sim_nodtu/mode=gc_bias/post_type=gibbs_nrep=100_tf=100/mean_cons/treeThr.RData")

suppressPackageStartupMessages(source("tree_helper_function.R"))
suppressPackageStartupMessages(source("brain_simulation_nodtu/mode=gc_bias/post_type=gibbs_nrep=100_tf=100/tree_analysis/tree_filter.R"))

In [13]:
txpDiff <- setdiff(treeMC0$tip.label,treeMC$tip.label)
length(txpDiff)

[1] 6547

In [14]:
infReps <- assays(yAggMeanInf0)[grep("infRep", assayNames(yAggMeanInf0))]
infReps <- abind::abind(as.list(infReps), along = 3)
lfc <- getLog2FC(infReps, colData(yAggMeanInf0)[["condition"]])
lfcMet <- abs(lfc/mcols(yAggMeanInf0)[["meanInfRV"]])

wlfcDiff <- sum(lfcMet[txpDiff])
wlfcDiff
l <- length(treeCons$tip)

[1] 15983.51

### Consensus Tree

In [4]:
dfLFC <- data.frame(matrix(nrow=0, ncol=3))
dCuts <- list()
colnames(dfLFC) <- c("Type", "Value", "nCuts")
l <- length(treeCons$tip)
lengths <- sapply(Descendants(treeCons, seq(length(treeCons$tip)+treeCons$Nnode)), length)
infReps <- assays(yAggCons)[grep("infRep", assayNames(yAggCons))]
infReps <- abind::abind(as.list(infReps), along = 3)
lfc <- getLog2FC(infReps, colData(yAggCons)[["condition"]])

lfcMet <- abs(lfc/mcols(yAggCons)[["meanInfRV"]])
globArr <- rep(-100, nrow(yAggCons))
val <- findMaxSum(treeCons, lfcMet, l+1, lengths)
optLfcMet <- globArr
cuts <- findCuts(treeCons, optLfcMet, lfcMet, l+1, lengths)
dfLFC <- rbind(dfLFC, data.frame(Type="Consensus", Value=val+wlfcDiff, nCuts = length(cuts)+length(txpDiff)))
dCuts[["Consensus"]] <- cuts

### Correlation Tree

In [5]:
l==length(treeCor$tip)
infReps <- assays(yAggCor)[grep("infRep", assayNames(yAggCor))]
infReps <- abind::abind(as.list(infReps), along = 3)
lfc <- getLog2FC(infReps, colData(yAggCor)[["condition"]])
lengths <- sapply(Descendants(treeCor, seq(length(treeCor$tip)+treeCor$Nnode)), length)
lfcMet <- abs(lfc/mcols(yAggCor)[["meanInfRV"]])
globArr <- rep(-100, nrow(yAggCor))
val <- findMaxSum(treeCor, lfcMet, l+1, lengths)
optLfcMet <- globArr
cuts <- findCuts(treeCor, optLfcMet, lfcMet, l+1, lengths)
dfLFC <- rbind(dfLFC, data.frame(Type="Anti-Correlation", Value=val + wlfcDiff, nCuts = length(cuts) + length(txpDiff)))
dCuts[["Anti-Correlation"]] <- cuts

[1] TRUE

### Mean Inf

In [10]:
l==length(treeMC$tip)
infReps <- assays(yAggMeanInf)[grep("infRep", assayNames(yAggMeanInf))]
infReps <- abind::abind(as.list(infReps), along = 3)
lfc <- getLog2FC(infReps, colData(yAggMeanInf)[["condition"]])
lfcMet <- abs(lfc/mcols(yAggMeanInf)[["meanInfRV"]])
globArr <- rep(-100, nrow(yAggMeanInf))
lengths <- sapply(Descendants(treeMC, seq(length(treeMC$tip)+treeMC$Nnode)), length)
val <- findMaxSum(treeMC, lfcMet, l+1, lengths)
optLfcMet <- globArr
cuts <- findCuts(treeMC, optLfcMet, lfcMet, l+1,lengths)
dfLFC <- rbind(dfLFC, data.frame(Type="MeanInf", Value=val+wlfcDiff, nCuts = length(cuts)+length(txpDiff)))
dCuts[["MeanInf"]] <- cuts

[1] TRUE

In [14]:
dfLFC

[1] 51685

Type,Value,nCuts
<chr>,<dbl>,<int>
Consensus,852574.3,41215
Anti-Correlation,710335.2,84051
MeanInf,617345.1,58232


### Mean Inf Thr0

In [15]:
# l==length(treeMC0$tip)
infReps <- assays(yAggMeanInf0)[grep("infRep", assayNames(yAggMeanInf0))]
infReps <- abind::abind(as.list(infReps), along = 3)
lfc <- getLog2FC(infReps, colData(yAggMeanInf0)[["condition"]])

lfcMet <- abs(lfc/mcols(yAggMeanInf0)[["meanInfRV"]])
lengths <- sapply(Descendants(treeMC0, seq(length(treeMC0$tip)+treeMC0$Nnode)), length)
globArr <- rep(-100, nrow(yAggMeanInf0))
val <- findMaxSum(treeMC0, lfcMet, length(treeMC0$tip)+1, lengths)
optLfcMet <- globArr
print(val)
cuts <- findCuts(treeMC0, optLfcMet, lfcMet, length(treeMC0$tip)+1, lengths)
dfLFC <- rbind(dfLFC, data.frame(Type="MeanInf0", Value=val, nCuts = length(cuts)))
dCuts[["MeanInf0"]] <- cuts

[1] 1020723


In [16]:
print(c(val, length(cuts)))

[1] 1020723   30757


### Consensus Thr

In [31]:
l==length(treeThr$tip)
infReps <- assays(yAggThrNS)[grep("infRep", assayNames(yAggThrNS))]
infReps <- abind::abind(as.list(infReps), along = 3)
lfc <- getLog2FC(infReps, colData(yAggThrNS)[["condition"]])

lfcMet <- abs(lfc/mcols(yAggThrNS)[["meanInfRV"]])
globArr <- rep(-100, nrow(yAggThrNS))
lengths <- sapply(Descendants(treeThr, seq(length(treeThr$tip)+treeThr$Nnode)), length)
val <- findMaxSum(treeThr, lfcMet, l+1,lengths)
optLfcMet <- globArr
cuts <- findCuts(treeThr, optLfcMet, lfcMet, l+1, lengths)
dfLFC <- rbind(dfLFC, data.frame(Type="Cons_Thr", Value=val+wlfcDiff, nCuts = length(cuts) + length(txpDiff)))
dCuts[["Cons_Thr"]] <- cuts

[1] TRUE

In [59]:
dfLFC

Type,Value,nCuts
<chr>,<dbl>,<int>
Consensus,852574.3,41215
Anti-Correlation,710335.2,84051
MeanInf,617345.1,58232
MeanInf0,1020723.0,30757
Cons_Thr,425170.4,6548


In [60]:
save(dCuts, file="environment/brain_sim_nodtu/mode=gc_bias/post_type=gibbs_nrep=100_tf=100/updated/comp_trees/lfcCuts_len.RData")

#### Consensus Tree

In [64]:
dfLFC <- data.frame(matrix(nrow=0, ncol=3))
dCuts <- list()
colnames(dfLFC) <- c("Type", "Value", "nCuts")
l <- length(treeCons$tip)
infReps <- assays(yAggCons)[grep("infRep", assayNames(yAggCons))]
infReps <- abind::abind(as.list(infReps), along = 3)
lfc <- getLog2FC(infReps, colData(yAggCons)[["condition"]])

lfcMet <- abs(lfc/mcols(yAggCons)[["meanInfRV"]])
globArr <- rep(-100, nrow(yAggCons))
val <- findMaxSum(treeCons, lfcMet, l+1)
optLfcMet <- globArr
cuts <- findCuts(treeCons, optLfcMet, lfcMet, l+1)
dfLFC <- rbind(dfLFC, data.frame(Type="Consensus", Value=val+wlfcDiff, nCuts = length(cuts)+length(txpDiff)))
dCuts[["Consensus"]] <- cuts

#### Correlation

In [65]:
l==length(treeCor$tip)
infReps <- assays(yAggCor)[grep("infRep", assayNames(yAggCor))]
infReps <- abind::abind(as.list(infReps), along = 3)
lfc <- getLog2FC(infReps, colData(yAggCor)[["condition"]])
lfcMet <- abs(lfc/mcols(yAggCor)[["meanInfRV"]])
globArr <- rep(-100, nrow(yAggCor))
val <- findMaxSum(treeCor, lfcMet, l+1)
optLfcMet <- globArr
cuts <- findCuts(treeCor, optLfcMet, lfcMet, l+1)
dfLFC <- rbind(dfLFC, data.frame(Type="Anti-Correlation", Value=val + wlfcDiff, nCuts = length(cuts) + length(txpDiff)))
dCuts[["Anti-Correlation"]] <- cuts

[1] TRUE

In [66]:
dfLFC

Type,Value,nCuts
<chr>,<dbl>,<int>
Consensus,210766.7,93762
Anti-Correlation,189507.6,118417


#### Mean Inf

In [17]:
l==length(treeMC$tip)
infReps <- assays(yAggMeanInf)[grep("infRep", assayNames(yAggMeanInf))]
infReps <- abind::abind(as.list(infReps), along = 3)
lfc <- getLog2FC(infReps, colData(yAggMeanInf)[["condition"]])
lfcMet <- abs(lfc/mcols(yAggMeanInf)[["meanInfRV"]])
globArr <- rep(-100, nrow(yAggMeanInf))
val <- findMaxSum(treeMC, lfcMet, l+1)
optLfcMet <- globArr
cuts <- findCuts(treeMC, optLfcMet, lfcMet, l+1)
dfLFC <- rbind(dfLFC, data.frame(Type="MeanInf", Value=val+wlfcDiff, nCuts = length(cuts)+length(txpDiff)))
dCuts[["MeanInf"]] <- cuts

In [16]:
dfLFC

Type,Value,nCuts
<chr>,<dbl>,<dbl>
Consensus,210766.7,93762
Anti-Correlation,189507.6,118417
MeanInf,210347.4,96183


#### MeanInf0

In [ ]:
# l==length(treeMC0$tip)
infReps <- assays(yAggMeanInf0)[grep("infRep", assayNames(yAggMeanInf0))]
infReps <- abind::abind(as.list(infReps), along = 3)
lfc <- getLog2FC(infReps, colData(yAggMeanInf0)[["condition"]])

lfcMet <- abs(lfc/mcols(yAggMeanInf0)[["meanInfRV"]])
globArr <- rep(-100, nrow(yAggMeanInf0))
val <- findMaxSum(treeMC0, lfcMet, length(treeMC0$tip)+1)
optLfcMet <- globArr
print(val)
cuts <- findCuts(treeMC0, optLfcMet, lfcMet, length(treeMC0$tip)+1)
dfLFC <- rbind(dfLFC, data.frame(Type="MeanInf0", Value=val, nCuts = length(cuts)))
dCuts[["MeanInf0"]] <- cuts

[1] 224250.6


#### Thr

In [ ]:
l==length(treeThr$tip)
infReps <- assays(yAggThrNS)[grep("infRep", assayNames(yAggThrNS))]
infReps <- abind::abind(as.list(infReps), along = 3)
lfc <- getLog2FC(infReps, colData(yAggThrNS)[["condition"]])

lfcMet <- abs(lfc/mcols(yAggThrNS)[["meanInfRV"]])
globArr <- rep(-100, nrow(yAggThrNS))
val <- findMaxSum(treeThr, lfcMet, l+1,lengths)
optLfcMet <- globArr
cuts <- findCuts(treeThr, optLfcMet, lfcMet, l+1)
dfLFC <- rbind(dfLFC, data.frame(Type="Cons_Thr", Value=val+wlfcDiff, nCuts = length(cuts) + length(txpDiff)))
dCuts[["Cons_Thr"]] <- cuts